In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import copy
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import backend as K
from scipy.io import loadmat
from scipy.spatial.distance import cosine as dcos
from scipy.spatial.distance import cosine as dcos


K.set_image_data_format( 'channels_last' )

Using TensorFlow backend.


In [2]:
def convblock(cdim, nb, bits=3):
    L = []
    
    for k in range(1,bits+1):
        convname = 'conv'+str(nb)+'_'+str(k)
        L.append( Convolution2D(cdim, kernel_size=(3, 3), padding='same', activation='relu', name=convname) )
    
    L.append( MaxPooling2D((2, 2), strides=(2, 2)) )
    
    return L

In [3]:
def vgg_face_blank():
    
    withDO = True
    
    if True:
        mdl = Sequential()
        
        mdl.add( Permute((1,2,3), input_shape=(224,224,3)) ) 

        for l in convblock(64, 1, bits=2):
            mdl.add(l)

        for l in convblock(128, 2, bits=2):
            mdl.add(l)
        
        for l in convblock(256, 3, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 4, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 5, bits=3):
            mdl.add(l)
        
        mdl.add( Convolution2D(4096, kernel_size=(7, 7), activation='relu', name='fc6') )
        if withDO:
            mdl.add( Dropout(0.5) )
        mdl.add( Convolution2D(4096, kernel_size=(1, 1), activation='relu', name='fc7') )
        if withDO:
            mdl.add( Dropout(0.5) )
        mdl.add( Convolution2D(2622, kernel_size=(1, 1), activation='relu', name='fc8') )
        mdl.add( Flatten() )
        mdl.add( Activation('softmax') )
        
        return mdl
    
    else:
        raise ValueError('not implemented')

In [4]:
facemodel = vgg_face_blank()

In [5]:
facemodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)      

In [6]:
if False:
    data = loadmat('vgg-face.mat', matlab_compatible=False, struct_as_record=False)
    l = data['layers']
    description = data['meta'][0,0].classes[0,0].description
else:
    data = loadmat('vgg_face_matconvnet/vgg_face.mat', matlab_compatible=False, struct_as_record=False)
    net = data['net'][0,0]
    l = net.layers
    description = net.classes[0,0].description

In [7]:
def copy_mat_to_keras(kmodel):

    kerasnames = [lr.name for lr in kmodel.layers]

    prmt = (0,1,2,3)

    for i in range(l.shape[1]):
        matname = l[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            l_weights = l[0,i][0,0].weights[0,0]
            l_bias = l[0,i][0,0].weights[0,1]
            f_l_weights = l_weights.transpose(prmt)
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])

In [8]:
copy_mat_to_keras(facemodel)

In [9]:
featuremodel = Model(inputs=facemodel.layers[0].input, outputs=facemodel.layers[-2].output)

In [10]:
featuremodel.save('self-trained models/neural_network_with_VGG_weigths.model')

featuremodel = load_model('self-trained models/neural_network_with_VGG_weigths.model')

C:\Users\rasmu\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [11]:
im1 = Image.open('vgg_face_matconvnet/Adrian_Nastase_0001.jpg')
im1 = im1.resize((224,224))
im1 = np.array(im1).astype(np.float32)
imarr1 = np.expand_dims(im1, axis=0)

im2 = Image.open('vgg_face_matconvnet/Adrian_Nastase_0002.jpg')
im2 = im2.resize((224,224))
im2 = np.array(im2).astype(np.float32)
imarr2 = np.expand_dims(im2, axis=0)

im3 = Image.open('vgg_face_matconvnet/AJ_Buckley.jpg')
im3 = im3.resize((224,224))
im3 = np.array(im3).astype(np.float32)
imarr3 = np.expand_dims(im3, axis=0)

In [13]:
fvec1 = featuremodel.predict(imarr1)[0,:]
fvec2 = featuremodel.predict(imarr2)[0,:]
fvec3 = featuremodel.predict(imarr3)[0,:]

dcos_1_2 = dcos(fvec1, fvec1)
print('1. Same picture. The Cosine distance between vectors 1 and 1 is:', dcos_1_2)

dcos_1_2 = dcos(fvec1, fvec2)
print('2. Same person. The Cosine distance between vectors 1 and 2 is:', dcos_1_2)

dcos_1_2 = dcos(fvec1, fvec3)
print('3. Different people. The Cosine distance between vectors 1 and 3 is:', dcos_1_2)

1. Same picture. The Cosine distance between vectors 1 and 1 is: 0.0
2. Same person. The Cosine distance between vectors 1 and 2 is: 0.17627805471420288
3. Different people. The Cosine distance between vectors 1 and 3 is: 0.5296575129032135
